In [1]:
%matplotlib inline
from smvjupyter2 import * #from smvjupyter import *     # import the UI

usfs_sites = "docs/usfs_sites/Sites_lf_geo.json"               # USFS sites
usfs_regions = "docs/usfs_admin/USFS_Regional_Boundaries.json" # admin regions

# plotter

In [6]:
def get_options(xrds, attribute):
    """ """
    
    options = []
    for name, xrda in xrds.items():
        isnull = xrda.isnull().data.all()
        isignore = name in ignore_variables
        if not any([isnull, isignore]):
            option = name if attribute=="dataset" else xrda.attrs[attribute]
            options.append(option)
            
    return(sorted(list(set(options))))
    

def get_checkboxes(xrds, group):
    """ """

    checkboxes = [Checkbox(
        description=str(o), 
        value=True, 
        indent=False,
        layout=Layout(width='auto')
    ) for o in get_options(xrds, group)]
    
    return(checkboxes)


def get_plot(xrds):
    """ """
    
    units = get_options(xrds, "units")
    sources = get_options(xrds, "source")
    
    fig, axs = plt.subplots(nrows=len(units), ncols=1, sharex=True, figsize=(20,12))
    plt.subplots_adjust(hspace=0.000)
    
    for i, u in enumerate(units):
        x = xrds.filter_by_attrs(units=u)
        ax = axs[i] if len(units)>1 else axs
        for n, d in x.items():
            d.mean("sample").plot.line(
                x="time", 
                label=n, 
                ax=ax, 
                add_legend=False)
        ax.set_xlabel(None)
        ax.set_ylabel(str(u))
        ax.set_title(None)
        ax.legend(loc=0, framealpha=1)

    plt.show()
            

class Plotter:
    """Generates the map/plot side-by-side widget container."""
    
    error = HTML("""<p>Please select one or more platform types.</p>""")
    
    groups = ["dataset", "source"]
    intervals = {"day": "d", "week": "w", "month": "m", "year": "y"}
    
    def __init__(self, layer, debug=False): 

        self.layer = layer
        self.debug = debug
        
        self.xr = layer.xr.sel(stat="Mean", drop=True)
        self.filtxr = layer.xr

        # selection container
        self.selected = dict(
            interval="day",
            stack=False,
            group="dataset",
            groupoptions={})
        
        # --------------------------------------------------------------------
        # selection ui
        
        # platform type options
        self.selected["type"] = get_options(self.xr, "type")
        self.platformtypes = Box([], layout=Layout(
            width="auto",
            display='flex',
            flex_flow='column',
            align_items='stretch'))
        test = []
        for t in self.selected["type"]:
            chkbx = Checkbox(
                value=True, 
                description=t, 
                indent=False, layout=Layout(width='auto'))
            chkbx.observe(self.type_handler, names="value")
            test.append(chkbx)
        self.platformtypes.children = test
        
        # interval options
        self.interval = RadioButtons(
            options=self.intervals.keys(), 
            value="day", 
            layout=Layout(width='auto'))
        self.interval.observe(self.options_handler, names="value")
        
        # dataset options      
        self.chkbxoutput = Box(
            children=[], 
            layout=Layout(
                height="200px",
                width="auto",
                display='flex',
                flex_flow='column',
                align_items='stretch',
                overflow_y="scroll",
                border="1px solid gray"))
        self.group_handler()

        # minimap ui
        #self.mapw = Map(                                             # map widget
        #    layers=(self.layero.layer, layer.points, bmap,), 
        #    center=(layer.lat, layer.lon), 
        #    zoom=zoom, width=mw)
        #self.output = Output(layout={"width": ow})                   # Output widget 
        #self.mapplotui = HBox([self.mapw, self.output])              # Map/plot ui        
        
        # selection ui (left panel) ---------------------- COMBINED
        self.selectui = Box([
            HTML("<b>Platform: </b>"),
            self.platformtypes,
            HTML("<b>Interval: </b>"),
            self.interval,
            HTML("<b>Datasets: </b>"),
            self.chkbxoutput
        ], layout=Layout(
            width="30%",
            display='flex',
            flex_flow='column',
            align_items='stretch',
            border="1px solid gray"))

        # --------------------------------------------------------------------
        # ui
        
        self.output = Output(layout={"border": "1px solid gray", "width": "auto"})
        self.ui = HBox([self.selectui, self.output])
        self.to_output()
        
    # ------------------------------------------------------------------------

    
    def type_handler(self, *args, **kwargs):
        """ """
        types = []
        for t in self.platformtypes.children:
            if t.value:
                types.append(t.description)
        print(types)
        self.selected["type"] = types
        self.group_handler()
        self.to_output()
    
    
    def group_handler(self, *args, **kwargs):
        """ """
        
        types = self.selected["type"]
        filtxr = self.xr.filter_by_attrs(type=lambda t: t in types)
        self.checkboxes = get_checkboxes(filtxr, "dataset")
        for c in self.checkboxes:
            c.observe(self.group_chk_handler, names="value")
        self.chkbxoutput.children = self.checkboxes
        
        
    def group_chk_handler(self, *args, **kwargs):
        """ """
        selections = []
        for c in self.checkboxes:
            if c.value:
                selections.append(c.description)
        self.selected["groupoptions"] = selections
        self.to_output()
    
    
    def options_handler(self, *args, **kwargs):
        """ """
        for n, w in {"interval": self.interval}.items():
            self.selected[n] = w.value
        self.to_output()
        
        
    def to_output(self, *args, **kwargs):
        """ """
        self.output.clear_output()
        
        types = self.selected["type"]
        
        interval = self.interval.value
        filtxr = self.xr.filter_by_attrs(type=lambda t: t in types)
        
        checked = self.selected["groupoptions"]
        filtxr = filtxr[checked]

        with self.output:
            
            units = get_options(filtxr, "units")
            sources = get_options(filtxr, "source")

            fig, axs = plt.subplots(
                nrows=len(units), 
                ncols=1, 
                sharex=True, 
                figsize=(15,10))
            plt.subplots_adjust(hspace=0.000)

            for i, u in enumerate(units):
                ax = axs[i] if len(units)>1 else axs
                x = filtxr.filter_by_attrs(units=u)
                
                if interval!="day":
                    iv = "1"+self.intervals[interval]
                    x = x.resample(time=iv).mean()

                for n, d in x.items():
                    d.mean("sample").plot.line(label=n, ax=ax, add_legend=False)

                ax.set_xlabel(None)
                ax.set_ylabel(str(u))
                ax.set_title(None)
                ax.legend(loc=0, framealpha=1)

            plt.show()

In [3]:
"""
def PlotDaymet(**args):
    """ """
    
    print(args)
    
daymet = layer.xr.filter_by_attrs(source="Daymet")
daymet.sel(stat="Mean").groupby("time.year").mean("sample").prcp.plot.step(where="mid", figsize=(15,8))
"""   

'\ndef PlotDaymet(**args):\n    \n    \n    print(args)\n    \ndaymet = layer.xr.filter_by_attrs(source="Daymet")\ndaymet.sel(stat="Mean").groupby("time.year").mean("sample").prcp.plot.step(where="mid", figsize=(15,8))\n'

In [9]:
layer = layers.iloc[1].layer
p = Plotter(layer)
p.ui

['in situ', 'spaceborne']
['spaceborne']
[]
['airborne']
['airborne', 'in situ']
['in situ']
['in situ', 'spaceborne']
['airborne', 'in situ', 'spaceborne']
['in situ', 'spaceborne']


# app

In [5]:
class JupyterSMV(object):
    """
    App.
    """

    head = HTML("""
    <h3>Soil Moisture Visualizer</h3>
    <p>Do such and such.</br>1. </br>a. </br>2. </br>b.</p>
    """)
    
    
    def __init__(self, primary=None, anc=None, nan=False, free=False):

        self.polys = LayerGroup()
        self.points = LayerGroup()
        self.apolys = LayerGroup()
        
        # -------------------------------------------------------------------
        
        self.mapw = Map(
            layers=(bmap, self.apolys, self.polys, self.points,), 
            center=(32.75, -109), 
            zoom=7, 
            width="auto", 
            height="auto",
            scroll_wheel_zoom=True)
        self.submit = Button(**submit_args)
        self.submit.on_click(self.submit_handler)
        self.progress = IntProgress(**progress_args)
        
        # -------------------------------------------------------------------
        
        if primary:                                   # if given, 
            self.load_features(primary)               # load input features
        if anc:
            self.load_ancillary(anc)
        dllayout = [self.mapw, HBox([self.submit, self.progress])]
        self.downloadui = VBox(dllayout)# + [HBox([self.out1, self.out2])])
        
        # -------------------------------------------------------------------
        
        self.plotui = Output()
        self.body = Accordion(
            children=[self.downloadui, self.plotui], 
            layout=Layout(width="auto", height="80%"))
        self.body.set_title(0, '1. Download')
        self.body.set_title(1, '2. Plot')
        self.ui = VBox([self.head,  self.body])

    def load_features(self, infeats):
        """ """
        features, cols = from_geojson(infeats)        # get features and cols

        layers = []                                   # a temporary list 
        for i, feat in enumerate(features):           # loop over features
            
            poly = Layer(i, feat, cols[i])            # get Layer class
            poly.layer.on_click(self.layer_click_handler)  # global callback
            self.polys.add_layer(poly.layer)          # add to poly grp

            pts, samps = LayerGroup(), []             # points group; Samples
            for j, p in enumerate(poly.ease):         # loop EASE grid pts
                s = Sample(j, p[0], p[1])             # make Sample instance
                pts.add_layer(s.pt)                   # add to points group
                samps.append((j, p[0], p[1], s))      # append tuple to list  

            samps = pd.DataFrame(samps, columns=sample_header)       # samples
            layers.append((i,poly.lat,poly.lon,poly,samps,pts,None)) # append
        
        self.layers = pd.DataFrame(layers, columns=layer_header)     # layers
        self.selected = None


    def load_ancillary(self, infeats):
        """ """          
        self.alayers = get_ancillary_data(infeats)
        for layer in self.alayers.layer:
            self.apolys.add_layer(layer)


    def layer_click_handler(self, **kwargs): 
        """ Evaluates when new polygon is selected. Layer.toggle first!"""
        
        if list(kwargs.keys()) != ['event', 'properties']: # check event
            return(None)                                   # skip basemap

        i = int(kwargs["properties"]["id"])             # get selected poly id
        layer_row = self.layers.iloc[i]                 # get row for selected
        layer_inst = layer_row.layer                    # get Layer class inst
        self.selected = i

        self.points.clear_layers()
        self.points.add_layer(layer_row["points"]) 
        self.mapw.center = (layer_inst.lat, layer_inst.lon)
        self.mapw.zoom = 9
        self.submit.disabled = True if layer_inst.dl else False
    
    
    def submit_handler(self, b):
        """Resets UI and sends requests to SMV when new submit."""
        self.submit.disabled = True
        
        i = self.selected
        layer_row = self.layers.iloc[i]
        sample = layer_row.samples["samp"].tolist()
        
        self.progress.max = len(sample)            # reset progress bar
        self.progress.min = 0                      
        self.progress.value = 0
        
        for s in sample:                           # loop over sample pts
            s.submit()                             # download the data
            self.progress.value += 1               # update progress bar
            s.update(**s.symbology)                # update style
        layer_row.layer.dl = True                  # set dl status to True
        
        xrds = xr.concat([s.xr for s in sample], "sample") # make xr dataset
        layer_row.layer.xr = xrds
        self.layers.at[self.selected,"xr"] = xrds 
        self.body.selected_index = 1
        
        layer_row.layer.getui()
        self.plotter = Plotter(layer_row.layer)
        self.plotui.clear_output()
        with self.plotui:
            display(self.plotter.ui)
        

smvdl = JupyterSMV(usfs_sites)#, anc=usfs_regions)#, nan=False)#,# init UI
smvdl.ui

## layers

In [8]:
layers = smvdl.layers 
layers

,id,lat,lon,layer,samples,points,xr
0,0,31.774734,-109.327589,<smvjupyter2.Layer object at 0x00000223B2E815C0>,id lat lon \ 0 0 32.08...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
1,1,31.505073,-110.536967,<smvjupyter2.Layer object at 0x00000223B2AF0EB8>,id lat lon \ 0 0 31.83...,LayerGroup(layers=(CircleMarker(color='#377eb8...,"[SoilSCAPE_surface, SoilSCAPE_rootzone, AirMOS..."
2,2,33.653808,-108.566810,<smvjupyter2.Layer object at 0x00000223B2AE1EF0>,id lat lon \ 0 0 33.84...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
3,3,35.298089,-111.641646,<smvjupyter2.Layer object at 0x00000223B378F860>,id lat lon \ 0 0 35.55...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
4,4,33.996685,-108.673179,<smvjupyter2.Layer object at 0x00000223B3542940>,id lat lon \ 0 0 34.17...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
5,5,34.020165,-109.401942,<smvjupyter2.Layer object at 0x00000223B35AEB70>,id lat lon \ 0 0 34.17...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
6,6,47.606681,-103.517068,<smvjupyter2.Layer object at 0x00000223B35C7AC8>,id lat lon \ 0 0 48.10...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
7,7,46.752166,-103.556410,<smvjupyter2.Layer object at 0x00000223B34EFEF0>,id lat lon \ 0 0 47.27...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
8,8,37.714688,-106.863892,<smvjupyter2.Layer object at 0x00000223B361BB70>,id lat lon \ 0 0 37.91...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None
9,9,37.337869,-103.076888,<smvjupyter2.Layer object at 0x00000223B4763588>,id lat lon \ 0 0 37.82...,LayerGroup(layers=(CircleMarker(fill_color='bl...,None


## plotter

In [ ]:
layer = layers.iloc[5].layer
p = Plotter(layer)
p.ui

In [ ]:
#dir(layers.iloc[0].layer)


py.iplot([{
    'x': t.index,
    'y': df[col],
    'name': col
}  for col in df.columns], filename='cufflinks/simple-line')

In [ ]:
radio_checkbox(plot_options)

# help-text-makers

## nice colors
1. use [`numpy.linspace`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) to make an array of evenly-spaced values between 0-1 
2. map values to **Set3** in [`matplotlib.cm`](https://matplotlib.org/api/cm_api.html) | [colormap reference](https://matplotlib.org/gallery/color/colormap_reference.html)
3. convert to hexadecimal with [`matplotlib.colors.to_hex`](https://matplotlib.org/api/_as_gen/matplotlib.colors.to_hex.html#matplotlib.colors.to_hex)

## the EASE Grid

The SMV returns data corresponding to 9x9-km cells within [the EASE grid system](https://nsidc.org/data/ease).

Here we select an array coordinates inside the boundary of the `shapely` geometry from before. The coordinates are then used to submit a series of requests for data from the SMV.
      
The EASE grid latitudes and longitudes are stored in two binary files:
* [docs/EASE2_M09km.lats.3856x1624x1.double](docs/EASE2_M09km.lats.3856x1624x1.double)
* [docs/EASE2_M09km.lats.3856x1624x1.double](docs/EASE2_M09km.lats.3856x1624x1.double)

Read the two files to arrays with [`numpy.fromfile`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fromfile.html), flatten them by calling [`numpy.ndarray.flatten`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.html), and stack them with [`numpy.dstack`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dstack.html) to make one very long 2-d array of latitude, longitude pairs:

In [ ]:
def get_ease(shapely_geom):
    """ """

    bnds = shapely_geom.bounds 
    ease = crds[
        (bnds[1]<lats) & (lats<bnds[3]) &     # ybnds < lat < ybnds
        (bnds[0]<lons) & (lons<bnds[2])]      # xbnds < lon < xbnds

    ease_reduced = []
    for p in ease:
        shapely_pt = shape({                  # input to shapely.shape is a
            "type": "Point",                  # python dict equivalent of
            "coordinates": (p[1], p[0])})     # geojson point geometry
        
        if shapely_geom.contains(shapely_pt): # if point inside poly
            ease_reduced.append([p[0], p[1]]) # return lat, lon tuple

    return(ease_reduced)

In [ ]:
smvdl = JupyterSMV(usfs_sites)      #, nan=False)#, anc=usfs_regions          # init UI

layers = app.layers
layer = layers.iloc[1]
xrds = layer.xr
nan = layer.layer.nan
  
plot_options = get_options(layer)

poly = layer["layer"].layer
pts = layer.points

mapw = Map(
    layers=(bmap,), 
    center=(layer["lat"], layer["lon"]), 
    zoom=7, 
    width="auto", 
    height="auto",
    scroll_wheel_zoom=True)
#mapw, unused_output = poly_mapper(layer)

# ---- ui

header = HTML("""
<h3>Soil Moisture Visualizer</h3>
<p>Do such and such.</br>1. </br>a. </br>2. </br>b.</p>
""")
ui = Accordion(children=[
    smvdl.ui, 
    radio_checkbox(plot_options),
    Text()], layout=Layout(width="auto", height="80%"))

ui.set_title(0, '1. Data')
ui.set_title(1, '2. Filters')
ui.set_title(2, '3. Details')

VBox([header,  ui])

In [ ]:

    
    """def draw(self):
        """ """
        self.output.clear_output()                               
        data = self.xrs
        
        By = self.bybutton.value
        if By is not "day":                                     # maybe aggregate
            data = data.groupby("time."+str(By)).mean() 
        
        with self.output:
            fig, axs = plt.subplots(nrows=self.nrow, ncols=self.ncol, **self.figure_args)

            # ax1: soil moisture
            for T in self.selected["type"]:
                D1 = data.filter_by_attrs(type=T)
                for Z in self.selected["soil_zone"]:
                    D2 = D1.filter_by_attrs(soil_zone=Z).sel(stat="Mean").mean(dim="sample")
                    self.tmpdf = calc_xrarrays(D2)
                    self.tmpdf.plot(x="Time", y="Mean", ax=axs[0])            
            axs[0].set_title("Mean soil moisture by platform type")                  # axis title
            axs[0].set_ylabel("m3/m3")
            
            # ax2: productivity
            data2 = data.filter_by_attrs(units="g m-2 d-1")
            for name, ds in data2.items():
                axs[1].set_title(ds.attrs["description"])                        # axis title
                tdata = ds.sel(stat="Mean")
                tmean, tstd = tdata.mean("sample"), tdata.std("sample")# get mean,std of sample dimension
                tmean.plot.line(label=name, ax=axs[1], add_legend=False, marker=None)         # plot a line
            axs[1].set_ylabel("g m-2 d-1")                                       # axis labels
            
            plt.show()"""

# plot testing

In [ ]:
import numpy as np
from pandas import date_range
import bqplot.pyplot as plt
from bqplot import *

fig = plt.figure()
axes_options = {'x': {'label': 'Date'}, 'y': {'label': 'Price'}}
line = plt.plot(
    dates, [security_1, security_2], 
    labels=['Security 1', 'Security 2'],
    axes_options=axes_options,
    display_legend=True)
fig